In [1]:
import numpy as np
import matplotlib.pyplot as plt


Things to include

- Solve Bang-Bang
- Solve Bang-SingularBang
- Plot trajectories
- Account for:
  - Mass of rocket
  - Reduction in mass of fuel due to expenditure
  - Air restistance drag D(h, v)
  - Max fuel expenditure


In [2]:
dry_mass = .6
init_fuel = .4

force_to_fuel_consumtion = .3 # Δm = -F/c

drag = lambda v,h = None
class Rocket(object):
    """
    Solves the Rocket Problem (Goddard Problem) of obtaining a maximimum height
     using a specified amount of fuel, with a drag as a function of height/velocity
     
    Params
    ------
    
    dry_mass : float
        descr
    fuel_mass : float
    drag : function
    f_max : float
    
    """
    def __init__(self, dry_mass, fuel_mass, drag, f_max=3.5):
        self.g = 9.8
        self.dry_mass = dry_mass
        self.init_fuel = init_fuel
        self.mass = dry_mass + init_fuel
        self.drag = drag
        self.f_max = f_max
        
        
    def _update(self, force, state):
        pass
        return state
    
    #return heights, velocities, control
    def solve_bsb(self):
        pass
    
    def solve_bb(self):
        pass
    
    def solve_fixedforce(self):
        pass
    
    def plot_trajectories(self):
        pass
    
    